
# Model-3 Random Forest Model

Exoplanet Exploration - Machine Learning Assignment
Summary
Create machine learning models capable of classifying candidate exoplanets from the raw dataset.

The algorithms used were:

* Model-1 - Logistic Refression
* Model-2 - Support Vector Machines (SVM) with Linear. Gaussian an dPolynomial kernels
* Model-3 - Random Forests
* Model-4 - Deep Model (Sequential)
* Model-5 - Decision Tree Model
Also, a hyper-parameter tuning with the meta-estimator function GridSearchCV() was used.

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\h_23m\anaconda3\envs\pythondata\lib\site-packages (0.0)


You should consider upgrading via the 'c:\users\h_23m\anaconda3\envs\pythondata\python.exe -m pip install --upgrade pip' command.


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

You should consider upgrading via the 'c:\users\h_23m\anaconda3\envs\pythondata\python.exe -m pip install --upgrade pip' command.


In [3]:
# Dependencies

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

### Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
pd.set_option('display.max_columns', None)
# Drop the null rows
df = df.dropna()
df.head(100)

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443,9.11,2.87,-1.62,25.8,2,5455,81,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638,39.30,31.04,-10.49,76.3,1,5853,158,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395,891.96,668.95,-230.35,505.6,1,5805,157,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406,926.16,874.33,-314.24,40.9,1,6031,169,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,0.762,0.139,-0.532,3.14020,0.06730,-0.06730,686.0,18.7,-18.7,2.77,0.90,-0.30,1160,427.65,420.33,-136.70,40.2,2,6046,189,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CONFIRMED,0,0,0,0,8.103633,2.030000e-05,-2.030000e-05,176.948600,0.002050,-0.002050,0.047,0.409,-0.047,3.16100,0.06350,-0.06350,887.9,24.5,-24.5,2.11,0.18,-0.20,687,52.82,16.30,-14.35,39.3,1,4861,131,-146,4.574,0.054,-0.041,0.732,0.062,-0.068,283.22141,40.421829,15.289
96,CONFIRMED,0,0,0,0,4.715108,1.940000e-05,-1.940000e-05,134.961780,0.003240,-0.003240,0.258,0.195,-0.258,2.50600,0.10300,-0.10300,417.5,22.0,-22.0,1.46,0.12,-0.14,823,108.84,33.58,-29.57,21.3,2,4861,131,-146,4.574,0.054,-0.041,0.732,0.062,-0.068,283.22141,40.421829,15.289
97,CONFIRMED,0,0,0,0,6.365840,8.060000e-06,-8.060000e-06,171.135750,0.001000,-0.001000,0.524,0.009,-0.460,2.97210,0.04080,-0.04080,780.7,13.1,-13.1,2.44,0.60,-0.21,914,165.33,121.73,-40.81,70.8,1,5776,155,-155,4.554,0.042,-0.168,0.849,0.212,-0.071,291.14951,40.420521,15.090
98,CONFIRMED,0,0,0,0,3.040330,3.370000e-07,-3.370000e-07,169.949011,0.000089,-0.000089,0.736,0.013,-0.024,1.84110,0.01680,-0.01680,10859.0,25.8,-25.8,8.99,0.80,-0.99,1021,257.29,86.59,-76.86,568.2,1,5035,151,-151,4.519,0.084,-0.063,0.791,0.071,-0.087,299.31610,40.822380,15.028


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

In [7]:
df.describe()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6.991000e+03,6.991000e+03,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6.991000e+03,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6.991000e+03,6.991000e+03,6.991000e+03,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.00000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000
mean,0.157059,0.244743,0.202975,0.125018,56.191248,1.851122e-03,-1.851122e-03,164.488820,0.009340,-0.009340,0.730631,1.945368,-0.337243,5.549368,0.320715,-0.320715,2.391190e+04,130.789186,-130.789186,121.194193,19.483454,-40.352509,1095.152768,8.263769e+03,3.599918e+03,-4.302621e+03,276.038764,1.233729,5700.682449,143.898155,-161.206980,4.305049,0.121091,-0.14048,1.740749,0.352710,-0.388568,292.082406,43.812143,14.271508
std,0.363882,0.429966,0.402243,0.330763,117.570962,7.184503e-03,7.184503e-03,67.020475,0.021989,0.021989,3.346956,9.401850,1.349036,6.425205,0.665352,0.665352,8.241240e+04,4689.233030,4689.233030,3519.737626,436.752064,1366.939375,856.234809,1.775255e+05,5.607270e+04,9.924848e+04,824.788481,0.650089,802.060405,46.566511,71.448481,0.439238,0.132048,0.08199,5.903415,0.839017,1.907797,4.762908,3.606167,1.350802
min,0.000000,0.000000,0.000000,0.000000,0.259820,1.100000e-08,-1.568000e-01,120.515914,0.000009,-0.569000,0.000000,0.000000,-59.320000,0.167000,0.000000,-20.200000,7.500000e+00,0.000000,-388600.000000,0.140000,0.000000,-77180.000000,92.000000,2.000000e-02,0.000000e+00,-5.600031e+06,1.600000,1.000000,2661.000000,0.000000,-1733.000000,0.047000,0.000000,-1.00700,0.109000,0.000000,-103.825000,279.856080,36.577381,6.966000
25%,0.000000,0.000000,0.000000,0.000000,2.620126,5.005000e-06,-2.401000e-04,132.683917,0.001145,-0.010000,0.198000,0.039900,-0.446000,2.446000,0.048900,-0.334000,1.604000e+02,9.400000,-48.000000,1.400000,0.240000,-2.080000,554.000000,2.227000e+01,1.139000e+01,-3.068950e+02,12.400000,1.000000,5297.000000,105.000000,-197.000000,4.209000,0.044000,-0.19500,0.829000,0.128000,-0.252000,288.704730,40.797760,13.455000
50%,0.000000,0.000000,0.000000,0.000000,8.947426,3.300000e-05,-3.300000e-05,136.739230,0.003990,-0.003990,0.539000,0.189800,-0.204000,3.793000,0.137000,-0.137000,4.245000e+02,20.200000,-20.200000,2.420000,0.520000,-0.310000,888.000000,1.466700e+02,8.090000e+01,-4.443000e+01,23.700000,1.000000,5757.000000,156.000000,-159.000000,4.436000,0.070000,-0.12700,0.999000,0.248000,-0.111000,292.314760,43.679661,14.534000
75%,0.000000,0.000000,0.000000,0.000000,34.282605,2.401000e-04,-5.005000e-06,169.937005,0.010000,-0.001145,0.891000,0.379000,-0.044000,6.197500,0.334000,-0.048900,1.533250e+03,48.000000,-9.400000,15.965000,2.650000,-0.140000,1397.500000,9.006700e+02,5.529550e+02,-6.400000e+00,82.250000,1.000000,6110.000000,174.000000,-112.000000,4.543000,0.149000,-0.08800,1.357000,0.357000,-0.069000,295.888550,46.693659,15.322000
max,1.000000,1.000000,1.000000,1.000000,1071.232624,1.568000e-01,-1.100000e-08,1472.522306,0.569000,-0.000009,100.806000,85.540000,0.000000,138.540000,20.200000,0.000000,1.541400e+06,388600.000000,0.000000,200346.000000,21640.000000,0.000000,14667.000000,1.094755e+07,3.617133e+06,0.000000e+00,9054.700000,8.000000,15896.000000,676.000000,0.000000,5.364000,1.472000,0.00000,180.013000,25.956000,0.000000,301.720760,52.336010,19.065000


### Select your features (columns)

In [8]:
# Set features. This will also be used as your x values.
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]

selected_features = df[['koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec',
                        'koi_period','koi_period_err1','koi_period_err2',
                        'koi_time0bk','koi_time0bk_err1','koi_time0bk_err2',
                        'koi_impact','koi_impact_err1','koi_impact_err2',
                        'koi_duration','koi_duration_err1','koi_duration_err2',
                        'koi_depth','koi_depth_err1','koi_depth_err2',
                        'koi_prad','koi_prad_err1','koi_prad_err2',
                        'koi_teq','koi_insol','koi_insol_err1','koi_insol_err2',
                        'koi_model_snr','koi_steff','koi_steff_err1','koi_steff_err2',
                        'koi_slogg','koi_slogg_err1','koi_slogg_err2',
                        'koi_srad','koi_srad_err1','koi_srad_err2',
                        'ra','dec','koi_kepmag']]

feature_names = selected_features.columns
selected_features.head(20)

# OR

# # Set target, features and feature_names.
# target = df["koi_disposition"]  (y)
# data = df.drop("koi_disposition", axis=1)  (=X)
# feature_names = data.columns
# data.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.5860,0.0590,-0.4430,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443,9.11,2.87,-1.62,25.8,5455,81,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,0.9690,5.1260,-0.0770,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638,39.30,31.04,-10.49,76.3,5853,158,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.2760,0.1150,-0.0920,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395,891.96,668.95,-230.35,505.6,5805,157,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,0.7010,0.2350,-0.4780,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406,926.16,874.33,-314.24,40.9,6031,169,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,0.7620,0.1390,-0.5320,3.14020,0.06730,-0.06730,686.0,18.7,-18.7,2.77,0.90,-0.30,1160,427.65,420.33,-136.70,40.2,6046,189,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
5,0,0,0,0,2.566589,1.780000e-05,-1.780000e-05,179.554370,0.004610,-0.004610,0.7550,0.2120,-0.5230,2.42900,0.16500,-0.16500,226.5,16.8,-16.8,1.59,0.52,-0.17,1360,807.74,793.91,-258.20,15.0,6046,189,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
6,0,0,0,0,16.068647,1.090000e-05,-1.090000e-05,173.621937,0.000517,-0.000517,0.0520,0.2620,-0.0520,3.53470,0.02410,-0.02410,4914.3,33.3,-33.3,5.76,0.22,-0.49,600,30.75,4.46,-6.66,161.9,5031,75,-83,4.485,0.083,-0.028,0.848,0.033,-0.072,286.99948,48.375790,15.841
7,0,0,0,0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,-0.000009,0.8180,0.0010,-0.0010,1.74319,0.00107,-0.00107,14231.0,4.2,-4.2,13.04,0.51,-0.51,1339,761.46,106.21,-95.76,4304.3,5820,78,-78,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,49.316399,11.338
8,0,1,0,0,2.204735,4.300000e-08,-4.300000e-08,121.358542,0.000016,-0.000016,0.2240,0.1590,-0.2160,3.88864,0.00203,-0.00203,6674.7,1.7,-1.7,16.10,0.81,-0.91,2048,4148.92,651.59,-654.46,5945.9,6440,76,-89,4.019,0.033,-0.027,1.952,0.099,-0.110,292.24728,47.969521,10.463
9,0,0,0,0,3.522498,1.980000e-07,-1.980000e-07,121.119423,0.000047,-0.000047,0.6310,0.0070,-0.0070,3.19843,0.00653,-0.00653,9145.7,6.6,-6.6,14.59,1.11,-1.11,1521,1264.67,307.23,-276.48,1741.5,6225,112,-137,4.169,0.055,-0.045,1.451,0.110,-0.110,281.28812,42.451080,13.563


In [9]:
X = selected_features.values
print(X.shape)

(6991, 39)


### Create a Train Test Split

Use `koi_disposition` for the y values

In [10]:

# get X and y variables
y = df["koi_disposition"]  # target
X = selected_features    # data

In [11]:
# label target values = koi_disposition column: 0 = 'CANDIDATE'; 1 = 'CONFIRMED'; 2 = 'FALSE POSITIVE'
target_labels = ["FALSE POSITIVE", "CONFIRMED", "CANDIDATE"]  
target_labels 

['FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE']

In [12]:
 
df["koi_disposition"].value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [13]:
# split the data to train and test values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [14]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6080,1,0,0,0,12.496435,2.213000e-04,-2.213000e-04,132.035800,0.014300,-0.014300,1.1700,0.078,-0.0850,84.32000,1.2000,-1.2000,271.7,2.5,-2.5,55.34,10.84,-25.30,1397,899.44,536.52,-676.12,141.5,6821,190,-286,3.805,0.390,-0.130,2.730,0.535,-1.248,289.23080,44.412483,13.054
3001,0,0,0,0,11.615625,1.528000e-04,-1.528000e-04,131.968430,0.008230,-0.008230,0.9770,0.007,-0.7050,2.23300,0.3620,-0.3620,191.2,28.0,-28.0,2.84,0.43,-1.00,905,158.69,65.40,-95.39,8.8,5332,88,-72,4.083,0.368,-0.092,1.453,0.218,-0.510,293.52756,41.111439,15.162
570,0,1,0,0,10.980246,6.930000e-07,-6.930000e-07,137.137607,0.000053,-0.000053,0.7330,0.001,-0.0010,3.74464,0.0034,-0.0034,50078.0,44.7,-44.7,21.94,5.82,-2.91,821,107.47,83.74,-34.92,1555.4,5952,159,-159,4.462,0.098,-0.182,0.897,0.238,-0.119,282.79764,43.578129,14.212
4897,1,0,0,0,466.908240,1.194000e-02,-1.194000e-02,136.373100,0.019000,-0.019000,0.0868,0.433,-0.0867,2.64000,0.6930,-0.6930,660.0,183.0,-183.0,2.19,1.13,-0.26,210,0.46,0.76,-0.14,5.4,5340,185,-146,4.456,0.102,-0.361,0.867,0.448,-0.103,297.65436,43.178551,15.202
625,0,1,1,1,1.061933,1.250000e-06,-1.250000e-06,133.850441,0.000978,-0.000978,0.7130,0.112,-0.2160,2.14290,0.0386,-0.0386,133.6,2.0,-2.0,2.29,0.48,-0.71,2508,9391.15,5840.09,-5394.87,80.2,6134,167,-167,3.975,0.259,-0.111,1.851,0.383,-0.575,288.90253,44.632992,12.953


### Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [15]:
X_train

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6080,1,0,0,0,12.496435,2.213000e-04,-2.213000e-04,132.035800,0.014300,-0.014300,1.1700,0.0780,-0.0850,84.32000,1.2000,-1.2000,271.7,2.5,-2.5,55.34,10.84,-25.30,1397,899.44,536.52,-676.12,141.5,6821,190,-286,3.805,0.390,-0.130,2.730,0.535,-1.248,289.23080,44.412483,13.054
3001,0,0,0,0,11.615625,1.528000e-04,-1.528000e-04,131.968430,0.008230,-0.008230,0.9770,0.0070,-0.7050,2.23300,0.3620,-0.3620,191.2,28.0,-28.0,2.84,0.43,-1.00,905,158.69,65.40,-95.39,8.8,5332,88,-72,4.083,0.368,-0.092,1.453,0.218,-0.510,293.52756,41.111439,15.162
570,0,1,0,0,10.980246,6.930000e-07,-6.930000e-07,137.137607,0.000053,-0.000053,0.7330,0.0010,-0.0010,3.74464,0.0034,-0.0034,50078.0,44.7,-44.7,21.94,5.82,-2.91,821,107.47,83.74,-34.92,1555.4,5952,159,-159,4.462,0.098,-0.182,0.897,0.238,-0.119,282.79764,43.578129,14.212
4897,1,0,0,0,466.908240,1.194000e-02,-1.194000e-02,136.373100,0.019000,-0.019000,0.0868,0.4330,-0.0867,2.64000,0.6930,-0.6930,660.0,183.0,-183.0,2.19,1.13,-0.26,210,0.46,0.76,-0.14,5.4,5340,185,-146,4.456,0.102,-0.361,0.867,0.448,-0.103,297.65436,43.178551,15.202
625,0,1,1,1,1.061933,1.250000e-06,-1.250000e-06,133.850441,0.000978,-0.000978,0.7130,0.1120,-0.2160,2.14290,0.0386,-0.0386,133.6,2.0,-2.0,2.29,0.48,-0.71,2508,9391.15,5840.09,-5394.87,80.2,6134,167,-167,3.975,0.259,-0.111,1.851,0.383,-0.575,288.90253,44.632992,12.953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3315,0,0,0,0,25.159339,4.236000e-04,-4.236000e-04,153.040400,0.011300,-0.011300,0.0350,0.4030,-0.0350,5.19800,0.2820,-0.2820,358.1,31.9,-31.9,1.76,0.53,-0.17,619,34.70,31.59,-10.63,13.1,5973,187,-229,4.506,0.050,-0.200,0.935,0.280,-0.093,294.48730,42.006920,15.759
2249,0,0,0,0,2.104722,8.860000e-07,-8.860000e-07,171.044219,0.000334,-0.000334,0.0010,0.3090,-0.0010,1.75470,0.0122,-0.0122,905.5,7.1,-7.1,2.89,0.79,-0.26,1365,821.44,654.25,-226.90,153.3,5703,154,-188,4.498,0.036,-0.204,0.975,0.264,-0.088,293.05722,45.551189,14.427
3751,0,1,0,1,43.798566,3.470000e-04,-3.470000e-04,172.585050,0.007470,-0.007470,1.1820,72.6400,-1.5900,6.40100,0.3100,-0.3100,290.4,13.5,-13.5,48.52,8.08,-17.50,721,63.80,35.13,-40.43,26.1,5611,186,-152,3.842,0.344,-0.108,2.227,0.371,-0.803,295.82376,44.795910,12.963
6016,0,0,0,0,91.069934,7.107000e-03,-7.107000e-03,196.391000,0.102000,-0.102000,0.5939,0.0065,-0.4992,17.33000,3.0000,-3.0000,12.7,1.9,-1.9,1.59,0.34,-0.57,1103,349.47,237.60,-228.61,8.7,9051,298,-379,3.623,0.305,-0.305,4.232,0.909,-1.521,295.51334,42.318790,9.748


In [16]:
# Scale your data using MinMaxScaler
# Import dependencies
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from tensorflow.keras.utils import to_categorical

# scale the data
X_minmax = MinMaxScaler().fit(X_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)

# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

print(encoded_y_test)

[0 2 2 ... 2 2 1]


### Train the Model ( Random Forest Model)



In [17]:

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train_minmax, encoded_y_train)
predictions = rf_model.predict(X_test_minmax)
rf_model

RandomForestClassifier()

In [18]:
# Trying another classification to see which features are more important
print(rf_model.feature_importances_)  # one value per feature

[0.0957382  0.06896507 0.11288937 0.03316601 0.02124816 0.01485691
 0.02136715 0.01404085 0.02261914 0.02874613 0.01911087 0.0102019
 0.01066841 0.0252729  0.03187325 0.02864059 0.02033231 0.01387446
 0.01454205 0.04310461 0.03477964 0.03388833 0.01567423 0.01616298
 0.01907493 0.01220908 0.05604641 0.0097319  0.02742369 0.02905004
 0.00956766 0.00917288 0.01059494 0.00947336 0.01050071 0.00921088
 0.01430133 0.01065882 0.01121985]


In [19]:
print(f"Random Forest - Training Data Score: {rf_model.score(X_train_minmax, encoded_y_train)}")
print(f"Random Forest - Testing Data Score: {rf_model.score(X_test_minmax, encoded_y_test)}")

Random Forest - Training Data Score: 1.0
Random Forest - Testing Data Score: 0.8930205949656751


In [20]:
sorted(zip(rf_model.feature_importances_, feature_names), reverse=True)

[(0.11288936805134354, 'koi_fpflag_co'),
 (0.09573820247596639, 'koi_fpflag_nt'),
 (0.06896506832059741, 'koi_fpflag_ss'),
 (0.05604640827511603, 'koi_model_snr'),
 (0.0431046118137374, 'koi_prad'),
 (0.03477964486876258, 'koi_prad_err1'),
 (0.03388832868900683, 'koi_prad_err2'),
 (0.033166008454890324, 'koi_fpflag_ec'),
 (0.03187325470493877, 'koi_duration_err1'),
 (0.029050035709332254, 'koi_steff_err2'),
 (0.028746129052266204, 'koi_time0bk_err2'),
 (0.02864058633915171, 'koi_duration_err2'),
 (0.02742368909614961, 'koi_steff_err1'),
 (0.025272895358535327, 'koi_duration'),
 (0.022619144996954128, 'koi_time0bk_err1'),
 (0.02136715068970857, 'koi_period_err2'),
 (0.02124816002539446, 'koi_period'),
 (0.02033231442449616, 'koi_depth'),
 (0.01911087409984799, 'koi_impact'),
 (0.019074926725360816, 'koi_insol_err1'),
 (0.016162980096677795, 'koi_insol'),
 (0.01567422787853448, 'koi_teq'),
 (0.014856908359007227, 'koi_period_err1'),
 (0.014542049722767611, 'koi_depth_err2'),
 (0.01430132

### Hyperparameter Tuning

Use `GridSearchCV` to tune the Random Forest model's parameters

In [21]:

# show current parameter
print(rf_model.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [22]:

# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [250, 300, 350],
              'max_depth': [125, 150, 175]}
grid = GridSearchCV(rf_model, param_grid, verbose=3)

In [23]:

# Convert the results to a Pandas data frame and make a short view of the results
# cvrf_results = pd.DataFrame(grid.cv_results_)
# cvrf_results_tiny = cvrf_results[['params', 'mean_test_score', 'std_test_score', 'rank_test_score', 'mean_train_score', 'std_train_score']]
# cvrf_results_tiny.sort_values(by='mean_test_score', ascending=False).head()

In [24]:
# Train the model with GridSearch
grid.fit(X_train_minmax, encoded_y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... max_depth=125, n_estimators=250, score=0.890, total=   2.9s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.889, total=   2.8s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.7s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.900, total=   2.8s
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.893, total=   2.8s
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.876, total=   2.8s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.891, total=   3.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.887, total=   3.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.901, total=   3.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.896, total=   3.4s
[CV] max_depth=125, n_estimators=300 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  2.5min finished


GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [125, 150, 175],
                         'n_estimators': [250, 300, 350]},
             verbose=3)

In [25]:
# Print the best parameters and score for this data set
print(f"Random Forest - Best Training Parameters: {grid.best_params_}")
print(f"Random Forest - Best Training Score: {grid.best_score_}")

Random Forest - Best Training Parameters: {'max_depth': 125, 'n_estimators': 300}
Random Forest - Best Training Score: 0.8909010035002437


In [26]:
# Training score:
grid.score(X_train_minmax, encoded_y_train)

1.0

In [27]:

# Testing score:
grid.score(X_test_minmax, encoded_y_test)

0.898741418764302

In [28]:
print(f"Random Forest - Training Grid Score: {grid.score(X_train_minmax, encoded_y_train)}")
print(f"Random Forest - Testing Grid Score: {grid.score(X_test_minmax, encoded_y_test)}")

Random Forest - Training Grid Score: 1.0
Random Forest - Testing Grid Score: 0.898741418764302


In [29]:

# Make prediction and save to variable for report.
predictions = grid.predict(X_test_minmax)
predictions

array([0, 2, 2, ..., 2, 2, 1])

In [30]:
# Print Classification Report.
from sklearn.metrics import classification_report
print(classification_report(encoded_y_test, predictions, target_names=target_labels))

                precision    recall  f1-score   support

FALSE POSITIVE       0.83      0.75      0.79       422
     CONFIRMED       0.81      0.84      0.82       450
     CANDIDATE       0.97      1.00      0.99       876

      accuracy                           0.90      1748
     macro avg       0.87      0.86      0.87      1748
  weighted avg       0.90      0.90      0.90      1748



In [31]:
# %matplotlib notebook
# from yellowbrick.classifier import ClassificationReport
# viz = ClassificationReport(RandomForestClassifier())
# viz.fit(X_train_minmax, encoded_y_train)
# viz.score(X_test_minmax, encoded_y_test)
# viz.finalize()
# viz.show(outpath="Images/randomforest_classificationreport.png")

In [32]:

# from yellowbrick.model_selection import FeatureImportances
# from yellowbrick.style import set_palette
# from yellowbrick.features import RadViz
# set_palette('yellowbrick')
# viz = FeatureImportances(rf, size=(500, 500))
# viz.fit(X_train_minmax, encoded_y_train)
# viz.show(outpath="Images/featureimportance.png")

### Save the Model

In [33]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'TanvirKhan_RandomForest.sav'
joblib.dump(rf_model, filename)

['TanvirKhan_RandomForest.sav']

In [34]:
### Loaded Model
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test_minmax, encoded_y_test)
result

0.8930205949656751

### Random Forest Classification - Comparison of Accuracy between using 40 features (all) vs. 10 features (threshold=0.03)


In [35]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.03
sfm = SelectFromModel(rf_model, threshold=0.03)

# Train the selector
sfm.fit(X_train_minmax, y_train)

SelectFromModel(estimator=RandomForestClassifier(), threshold=0.03)

In [36]:

# Print the names of the most important features
for feature_list_index in sfm.get_support(indices=True):
    print(feature_names[feature_list_index])

koi_fpflag_nt
koi_fpflag_ss
koi_fpflag_co
koi_fpflag_ec
koi_duration_err1
koi_duration_err2
koi_prad
koi_prad_err1
koi_model_snr
koi_steff_err2


In [37]:

# Transform the data to create a new dataset containing only the most important features
X_important_train = sfm.transform(X_train_minmax)
X_important_test = sfm.transform(X_test_minmax)

In [38]:
# Create a new random forest classifier for the most important features
clf_important = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

# Train the new classifier on the new dataset containing the most important features
clf_important.fit(X_important_train, encoded_y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [39]:

# Apply The Full Featured Classifier To The Test Data
y_pred = rf_model.predict(X_test_minmax)

# View The Accuracy Of Our Full Feature (40 Features) Model
print(f"Random Forest - Accuracy Score 40 features: {accuracy_score(encoded_y_test, y_pred)}")

Random Forest - Accuracy Score 40 features: 0.8930205949656751


In [40]:
# Apply The Full Featured Classifier To The Test Data
y_important_pred = clf_important.predict(X_important_test)

# View The Accuracy Of Our Limited Feature Model
print(f"Random Forest - Accuracy Score 10 features: {accuracy_score(encoded_y_test, y_important_pred)}")

Random Forest - Accuracy Score 10 features: 0.8975972540045767


In [43]:
# By removing less importance features doesn't show that much improvements on accuracy.
# Accuracy = 0.8930205949656751 - 0.8975972540045767
# Accuracy


-0.004576659038901587

In [42]:
# Accuracy improvement is only 1.8%